In [ ]:
# %%bash
# export SPEDAS_DATA_DIR="/mnt/d/data/pyspedas-data"
# echo $SPEDAS_DATA_DIR

In [ ]:
from datetime import datetime
import string
import dateutil.parser

import asilib
import asilib.map
import asilib.asi
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import matplotlib.patches
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import IRBEM
import cdflib
import pyspedas

import elfinasi

In [ ]:
print(f'asilib version: {asilib.__version__}, elfinasi version: {elfinasi.__version__}, cdflib version: {cdflib.__version__}, pyspedas version: {pyspedas.version()}')

14-Dec-25 13:35:31: pyspedas version: 1.7.0


asilib version: 0.27.0, elfinasi version: 0.0.2, cdflib version: 1.3.7, pyspedas version: None


In [ ]:
# elfin_probe = 'a'
# time_range_str = ('2022-03-05T14:52', '2022-03-05T14:56')

# elfin_probe = 'b'
# time_range_str = ('2022-03-05T15:29', '2022-03-05T15:34')
# time_range_str = ('2022-03-05T15:00', '2022-03-05T16:00')

elfin_probe = 'a'
time_range_str = ('2022-09-04T04:15', '2022-09-04T04:25')

time_range = (dateutil.parser.parse(time_range_str[0]), dateutil.parser.parse(time_range_str[1]))

In [ ]:
def map_elfin(df, alt=110, hemi_flag=0):
    """
    Map ELFIN's location along the magnetic field line to alt using IRBEM.MagFields.find_foot_print.

    Parameters
    ----------
    alt: float
        The mapping altitude in units of kilometers
    hemi_flag: int
        What direction to trace the field line: 
        0 = same magnetic hemisphere as starting point
        +1   = northern magnetic hemisphere
        -1   = southern magnetic hemisphere
        +2   = opposite magnetic hemisphere as starting point
    """
    m = IRBEM.MagFields(kext='T89')
    _all = np.zeros_like(df.loc[:, ['alt', 'lat', 'lon']])

    for i, (time, row) in enumerate(df.iterrows()):
        X = {'Time':time, 'x1':row['alt'], 'x2':row['lat'], 'x3':row['lon']}
        _all[i, :] = m.find_foot_point(X, {'Kp':56}, alt, hemi_flag)['XFOOT']
    _all[_all == -1E31] = np.nan
    mapped_df = df.copy()
    mapped_df.loc[:, ['alt', 'lat', 'lon']] = _all
    return mapped_df

In [ ]:
pyspedas.projects.elfin.state(time_range_str, probe=elfin_probe.lower())

14-Dec-25 13:35:32: Downloading remote index: https://data.elfin.ucla.edu/ela/l1/state/defn/2022/
14-Dec-25 13:35:32: File is current: elfin_data/ela/l1/state/defn/2022/ela_l1_state_defn_20220904_v02.cdf
14-Dec-25 13:35:32: No VAR_TYPE or PARAMETER_TYPE attributes defined for variable ela_sun, skipping
14-Dec-25 13:35:32: time_clip: ela_pos_gei has no data in requested range
14-Dec-25 13:35:32: time_clip: ela_vel_gei has no data in requested range


['ela_pos_gei',
 'ela_vel_gei',
 'ela_att_gei',
 'ela_att_solution_date',
 'ela_att_flag',
 'ela_att_spinper',
 'ela_spin_orbnorm_angle',
 'ela_spin_sun_angle']

In [ ]:
pyspedas.data_quants[f'el{elfin_probe.lower()}_pos_gei'].time

<xarray.DataArray 'time' (time: 86400)> Size: 691kB
array(['2159-10-17T06:01:43.694951322', '2119-11-14T07:35:47.600726938',
       '2095-02-27T12:52:10.318987635', ..., '2014-11-09T02:38:44.786342041',
       '2252-12-08T23:45:24.675510425', '2177-02-06T03:14:49.477970073'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 691kB 2159-10-17T06:01:43.694951322 ... 21...

In [ ]:
state = elfinasi.State(elfin_probe, time_range[0])

In [ ]:
cdflib.cdfepoch.to_datetime(state.state_obj.varget(f'el{elfin_probe.lower()}_state_time'))

array(['2159-10-17T06:01:43.694951322', '2119-11-14T07:35:47.600726938',
       '2095-02-27T12:52:10.318987635', ...,
       '2014-11-09T02:38:44.786342041', '2252-12-08T23:45:24.675510425',
       '2177-02-06T03:14:49.477970073'], dtype='datetime64[ns]')

In [ ]:
state['epoch']

array(['2159-10-17T06:01:43.694951322', '2119-11-14T07:35:47.600726938',
       '2095-02-27T12:52:10.318987635', ...,
       '2014-11-09T02:38:44.786342041', '2252-12-08T23:45:24.675510425',
       '2177-02-06T03:14:49.477970073'], dtype='datetime64[ns]')

In [ ]:
pad_obj_nflux = elfinasi.EPD_PAD(
    elfin_probe, time_range, start_pa=0, min_counts=None, accumulate=1, spin_time_tol=(2.5, 12),
    lc_exclusion_angle=0
)

14-Dec-25 13:35:32: https://cdaweb.gsfc.nasa.gov/WS/cdasr/1/dataviews/sp_phys/datasets failed with http code 404
14-Dec-25 13:35:32: data_request = <cdasws.datarequest.CdfRequest object at 0x7191f2402f90>
14-Dec-25 13:35:32: response.text: <?xml version="1.0" encoding="UTF-8" standalone="yes"?><DataResult xmlns="http://cdaweb.gsfc.nasa.gov/schema" xmlns:ns2="http://cdf.gsfc.nasa.gov"><Message>No data available.</Message><Status>No data found for OMNI2_H0_MRG1HR, time range: 2159-10-17T06:01:43 - 2177-02-06T03:14:49</Status><Error>No data available.</Error></DataResult>
14-Dec-25 13:35:32: get_data_result failed with http code 404
14-Dec-25 13:35:32: data_request = <cdasws.datarequest.CdfRequest object at 0x7191f2402f90>


[datetime.datetime(2159, 10, 17, 6, 1, 43, 694951)
 datetime.datetime(2119, 11, 14, 7, 35, 47, 600726)
 datetime.datetime(2095, 2, 27, 12, 52, 10, 318987) ...
 datetime.datetime(2014, 11, 9, 2, 38, 44, 786342)
 datetime.datetime(2252, 12, 8, 23, 45, 24, 675510)
 datetime.datetime(2177, 2, 6, 3, 14, 49, 477970)] 2159-10-17T06:01:43.694951+00:00 2177-02-06T03:14:49.477970+00:00


14-Dec-25 13:35:32: /home/mike/research/elfinasi/src/elfinasi/elfin.py:1033: SyntaxWarning: "\c" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\c"? A raw string is also an option.
  ax.set(ylim=(0, 180), facecolor='grey', ylabel=f'Pitch angle\n[$\circ$]')

14-Dec-25 13:35:32: /home/mike/research/elfinasi/src/elfinasi/elfin.py:1098: SyntaxWarning: "\c" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\c"? A raw string is also an option.
  ax.set(ylim=(0, 180), facecolor='grey', ylabel=f'Pitch angle\n[$\circ$]')

14-Dec-25 13:35:32: /home/mike/research/elfinasi/src/elfinasi/elfin.py:1230: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
  ax.text(0.01, 0.98, f'$\sigma_{{BLC/DLC}}$', transform=ax.transAxes, va='top', fontsize=15, color='white')

14-Dec-25 13:35:32: /home/mike/research/elfinasi/src/elfinasi/el

TypeError: 'NoneType' object is not subscriptable